<a href="https://colab.research.google.com/github/Nethmifonseka97/Player_Analysis/blob/main/Player_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install flask-ngrok


# install dependencies
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git' 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__

!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# You may need to restart your runtime prior to this, to let your installation take effect

# Some basic setup

# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

!pip install webcolors==1.3
# import some common libraries
import numpy as np
import cv2
import random
from webcolors import name_to_rgb
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog


!wget http://ai-analytics.asia/img/s5.png -O input.jpg

!git clone https://github.com/facebookresearch/detectron2 detectron2_repo

!pip install -e detectron2_repo
from shapely.geometry import Point, Polygon
import pandas as pd
import csv


# Python code for Multiple Color Detection
import numpy as np
import cv2


from flask import Flask
from flask import request, url_for, redirect, render_template, jsonify
from flask_ngrok import run_with_ngrok



Requirement already up-to-date: torch in /usr/local/lib/python3.7/dist-packages (1.8.1+cu101)
Requirement already up-to-date: torchvision in /usr/local/lib/python3.7/dist-packages (0.9.1+cu101)
Requirement already up-to-date: cython in /usr/local/lib/python3.7/dist-packages (0.29.23)
  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-1ckji1i7
  Running command git clone -q https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-1ckji1i7
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-8h8au8dx
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-8h8au8dx
     |████████████████████████████████| 645kB 11.1MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5-cp37-none-any.whl size=64231 sha256=7c00309ff970c1eebaeb3cb34f15a59a9905f439a24f42a9948983f8c239aef0
  Stored in directory: /tmp/pip-ephem-wheel-cache-bkn14dtf/wheels/48/53/79/3c6485543a4455a0006f5db590ab9957622b6227011941de06
  

In [5]:

import requests
#app = Flask(__name__)
#run_with_ngrok(app)  

app = Flask(__name__, template_folder='/content/drive/My Drive/basketball/Project 3/')
run_with_ngrok(app)  
@app.route('/')
def home():
    return render_template("index.html")
 
@app.route('/game_analysis', methods=['POST'])
def game_analysis():
  try:
    get_color = [x for x in request.form.values()]
    print (get_color)

    path = get_color[2]
    print (path)

  
    #im = cv2.imread("./input.jpg")
    #im = cv2.imread("./a1.png")
    #cv2_imshow(im)


      
    # Four corners of the 3D court 
    # Start top-left corner and go anti-clock wise
    src_pts = np.array([
        [320,330],
            [1100,330],
            [1300,550],
            [-10,550],

            
        ])    
    src_pts2 = np.array([
        [320,280],
            [1100,330],
            [1340,560],
            [-40,520],
            
        ]) 



        
    video_id = 1 
    team_id = ""
    video_url = path 
    boundaries = src_pts
    #im_poly = im.copy()
    # cv2.fillPoly(img_src, [src_pts], 255)
    #cv2.polylines(im_poly, [src_pts], isClosed=True, color=[255,0,0], thickness=2)


    # Use the boxes info from the tensor prediction result
    #
    # x1,y1 ------
    # |          |
    # |          |
    # |          |
    # --------x2,y2
    #

    '''![picture](https://drive.google.com/file/d/1mLyJuMSsGlLeQ5Tajqepav5xNOOWsK-Y/uc?usp=sharing)


    !wget http://ai-analytics.asia/img/s5.png -O input.jpg
    im = cv2.imread("./a1.png")'''

    #!wget http://ai-analytics.asia/img/s5.png -O input.jpg
    #im = cv2.imread("./input.jpg")

    cfg = get_cfg()
    #cfg.merge_from_other_cfg
    cfg.merge_from_file("./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
    cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
    predictor = DefaultPredictor(cfg)

    def drawPlayers(im, pred_boxes, showResult=False):


      
      players_output = predictor(im)

      
      # look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
      instances = players_output["instances"]
      pred_boxes = instances.get("pred_boxes")
      pred_classes = instances.get("pred_classes")

      
      # We can use `Visualizer` to draw the predictions on the image.
      v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.0)
      v = v.draw_instance_predictions(players_output["instances"].to("cpu"))


      


      data = [[10,10],[12,12]]

      pos = pd.DataFrame(data,columns=['pos', 'time'])


      import time

      t = time.process_time()
      #elapsed_time = time.process_time() - t


      #rgb_color1 = input('Enter team colours ')
      rgb_color1 = name_to_rgb(get_color[0])
      #print(rgb_color1)
      c1 = np.uint8([[rgb_color1]]) #here insert the bgr values which you want to convert to hsv
      hsvcolour1 = cv2.cvtColor(c1, cv2.COLOR_BGR2HSV)
      lowerLimit1 = hsvcolour1[0][0][0] + 40, 80, 110
      upperLimit1 = hsvcolour1[0][0][0] + 70, 255, 255


      #get the second team colour
      rgb_color2 = name_to_rgb(get_color[1])
      #print(rgb_color2)
      c2 = np.uint8([[rgb_color2]]) #here insert the bgr values which you want to convert to hsv
      hsvcolour2 = cv2.cvtColor(c2, cv2.COLOR_BGR2HSV)
      lowerLimit2 = hsvcolour2[0][0][0] + 0, 0, 231
      upperLimit2 = hsvcolour2[0][0][0] + 180, 18, 255




      # Convert the imageFrame in
      # BGR(RGB color space) to
      # HSV(hue-saturation-value)
      # color space
      hsvFrame = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)

      #cv2_imshow(hsvFrame)

      # Set range for red color and
      # define mask
      #red_lower = np.array([136, 87, 111], np.uint8)
      #red_upper = np.array([180, 255, 255], np.uint8)

      lowerLimit1 = np.array(lowerLimit1, np.uint8)
      upperLimit1 = np.array(upperLimit1, np.uint8)
      print (lowerLimit1, upperLimit1 )
      team_mask1 = cv2.inRange(hsvFrame, lowerLimit1, upperLimit1)


      lowerLimit2 = np.array(lowerLimit2, np.uint8)
      upperLimit2 = np.array(upperLimit2, np.uint8)
      print (lowerLimit2, upperLimit2 )
      team_mask2 = cv2.inRange(hsvFrame, lowerLimit2, upperLimit2)



      # Morphological Transform, Dilation
      # for each color and bitwise_and operator
      # between imageFrame and mask determines
      # to detect only that particular color
      kernal = np.ones((5, 5), "uint8")

      # For red color
      team_mask1 = cv2.dilate(team_mask1, kernal)
      res_red = cv2.bitwise_and(im, im,
                  mask = team_mask1)

      # For black color
      team_mask2 = cv2.dilate(team_mask2, kernal)
      res_black = cv2.bitwise_and(im, im,
                  mask = team_mask2)




      f = open('ab.txt', 'a')
      color1 = [0, 0, 255] 
      color2 = [0, 0, 0] 
      thickness = 5
      radius = 5

      

      i  = 0
      for box in pred_boxes:
        
        # Include only class Person
        if pred_classes[i] == 0:  
            
          x1 = int(box[0])
          y1 = int(box[1])
          x2 = int(box[2])
          y2 = int(box[3])

          xc = x1 + int((x2 - x1)/2)
          player_pos1 = (xc - 1, y2)
          player_pos2 = (xc + 1, y2 + 1)

          court = Polygon(src_pts)
          court2 = Polygon(src_pts2)

          # Draw only players that are within the basketball court
          if Point(player_pos1).within(court):
            if showResult:
              print(".")
            
            # Creating contour to track red color
            contours, hierarchy = cv2.findContours(team_mask1,
                      cv2.RETR_TREE,
                      cv2.CHAIN_APPROX_SIMPLE)
            for pic, contour in enumerate(contours):
              area = cv2.contourArea(contour)
              if (area > 300):
                x, y, w, h = cv2.boundingRect(contour)

                if Point((x,y)).within(court2):
                  #cv2.rectangle(im, (x,y),(x + w, y + h),  (0, 0, 255), 2)

                  if (x== x1) : 
                    print(x,x1)
                    cv2.rectangle(im, (x,y),(x + w, y + h), (0, 0, 255), 2) 
                    team_id = "a" 
                    #cv2.rectangle(im, player_pos1,player_pos2, (0, 0, 255), thickness)
                    #cv2.putText(im, "T1", (x,y),cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 0, 255))
                    elapsed_time = time.process_time() - t
                    print("Pos1 " + str(player_pos1) + "Pos2"+ str(player_pos2)+ "time "+str(elapsed_time)+ "color "+str(rgb_color1), file = f)
          
                  
                    
            # Creating contour to track black color
            contours, hierarchy = cv2.findContours(team_mask2,
              cv2.RETR_TREE,
              cv2.CHAIN_APPROX_SIMPLE)
            for pic, contour in enumerate(contours):
              area = cv2.contourArea(contour)
              if (area > 300):
                p, q, r, s = cv2.boundingRect(contour)
                #print(p,q,r,s)
                if Point((p,q)).within(court2):
                  if (p == x1) :
                    print(p,x1) 
                    cv2.rectangle(im, (p,q),(p + r, q + s), (255, 255, 255), 2) 
                    team_id = "b" 
                    #cv2.rectangle(im, player_pos1, player_pos2, (255, 0, 0), thickness)
                    #cv2.putText(im, "T2", player_pos1,cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255, 255, 255))
                    elapsed_time = time.process_time() - t
                    print("Pos1 " + str(player_pos1) + "Pos2"+ str(player_pos2)+ "time "+str(elapsed_time)+ "color "+str(rgb_color2), file = f)
              

                    
            #elapsed_time = time.process_time() - t
            #print("Pos1 " + str(player_pos1) + "Pos2"+ str(player_pos2)+ "time "+str(elapsed_time), file = f)
          
            pos.append({'pos': player_pos1[0],'time': player_pos1[1]},ignore_index=True)
            #print(player_pos2)

            i = i + 1    

          
      
      
    
      if showResult:
        print ("<")
        #cv2_imshow(im)



      pos.to_csv('pos1.csv', index=False)
      f.close()




    from google.colab import files
    #files.download('ab.txt') 


    !wget http://ai-analytics.asia/img/s2.jpg -O court.jpg

    img_dst = cv2.imread('./court.jpg')

    # Four corners of the court + mid-court circle point in destination image 
    # Start top-left corner and go anti-clock wise + mid-court circle point
    dst_pts = np.array([
        [15,20],
            [600,20],
            [600,380],
            [15,380],
            
        
        ])   

    cv2.polylines(img_dst, [dst_pts], isClosed=True, color=[255,0,0], thickness=2)
    #cv2_imshow(img_dst)


    def homographyTransform(im, showResult=False):

      # Calculate Homography
      h, status = cv2.findHomography(src_pts, dst_pts)
      img_out = cv2.warpPerspective(im, h, (img_dst.shape[1], img_dst.shape[0]))
      
      if showResult:
        print ('//')
        #cv2_imshow(img_out)

      return img_out  
    # Try out
    #img_out = homographyTransform(im, True)  


    def getPlayersMask1(im):
      lower_range1 = np.array([0,0,255])                         # Set the Lower range value of blue in BGR
      upper_range1 = np.array([0,0,255])                     # Set the Upper range value of blue in BGR
      mask1 = cv2.inRange(im, lower_range1, upper_range1)     # Create a mask with range
      result1 = cv2.bitwise_and(im, img_out, mask = mask1)   # Performing bitwise and operation with mask in img variable
      #cv2_imshow(result)                              

      return cv2.inRange(result1, lower_range1, upper_range1)  
      

    def getPlayersMask2(im):
      lower_range2 = np.array([255,0,0])                         # Set the Lower range value of blue in BGR
      upper_range2 = np.array([255,0,0])                   # Set the Upper range value of blue in BGR
      mask2 = cv2.inRange(im, lower_range2, upper_range2)   # Create a mask with range
      result2 = cv2.bitwise_and(im, img_out, mask = mask2)  # Performing bitwise and operation with mask in img variable

      # cv2_imshow(result)                              
      return cv2.inRange(result2,lower_range2, upper_range2)  



    def drawPlayersOnCourt(im, coord, color, radius=10):
      for pos in coord:
        center_coordinates = (pos[0], pos[1])
        cv2.circle(im, center_coordinates, radius, color, thickness=-1) 

      return im




    # Draft method to draw lines between history player positions to show trail
    def drawCoordinateLines(result1,resul2, pts, currentFrame, player):
      
      for i in np.arange(1, len(pts)):
        
        # if either of the tracked points are None, ignore them
        if pts[i - 1] is None or pts[i] is None:
          continue

        thickness = int(np.sqrt(30 / float(i + 1)) * 2.5)
        print("player=%s" %player)
        x1 = pts[i - 1][0][0]
        x2 = pts[i - 1][0][1]
        print("x1=%d, x2=%d" %(x1, x2))
        y1 = pts[i][0][0]
        y2 = pts[i][0][1]
        print("y1=%d, y2=%d" %(y1, y2))
        print(" ---------------------- ")
        cv2.line(result1, (x1, x2), (y1, y2), red_color, thickness)
        cv2.line(result2, (x1, x2), (y1, y2), white_color, thickness)

      return result1,result2



    import time
    import progressbar
    from time import sleep
    from collections import deque
    from detectron2.utils.visualizer import ColorMode
    from detectron2.utils.visualizer import GenericMask
    import imutils

    #!wget /content/drive/My Drive/basketball/Project 3/abc.mp4 -O video.mp4
    #vs = cv2.VideoCapture(path)
    !wget http://ai-analytics.asia/vid/abc.mp4 -O video.mp4
    vs = cv2.VideoCapture("./video.mp4")
    #vs = cv2.VideoCapture("")
    totalFrames = int(vs.get(cv2.CAP_PROP_FRAME_COUNT))

    grabbed = True
    currentFrame = 0
    start = time.time()
    writer = None

    bar = progressbar.ProgressBar(maxval=totalFrames, \
          widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

    bar.start()

    court_img = cv2.imread('court.jpg')
    cv2_imshow(court_img)
    white_color = (255,255,255)
    red_color = (0,0,255)


    # loop over frames from the video file stream (207)
    while grabbed and currentFrame<=5 :     
      
      # read the next frame from the file
      (grabbed, frame) = vs.read()

      #if writer is None:
        #fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        #writer = cv2.VideoWriter("output.mp4", fourcc, 24, (court_img.shape[1], court_img.shape[0]), True)

      if grabbed:
        print(currentFrame)

       
        players_output = predictor(frame)

      
        # Get player positions
        outputs = predictor(frame)  
        instances = outputs["instances"].to("cpu")
        boxes = instances.get("pred_boxes")

        court = court_img.copy()

        # Draw players on video frame
        drawPlayers(frame, boxes, False)
        
        img_out = homographyTransform(frame, False)
        #cv2_imshow(img_out)

        mask1 = getPlayersMask1(img_out)
        mask2 = getPlayersMask2(img_out)
        #cv2_imshow(mask)

        # Get the contours from the players "dots" so we can reduce the coordinates
        # to the number of players on the court.
        cnts1 = cv2.findContours(mask1.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts1 = imutils.grab_contours(cnts1)

        cnts2 = cv2.findContours(mask2.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts2 = imutils.grab_contours(cnts2)
      
        if cnts1 is not None:      
          for cnt1 in cnts1:
            result1 = drawPlayersOnCourt(court, cnt1[0], red_color)
            cv2_imshow(result1)

        if cnts2 is not None:      
          for cnt2 in cnts2:
            result2 = drawPlayersOnCourt(court, cnt2[0], white_color)
            cv2_imshow(result2)

        #cv2_imshow(result1)  
        #cv2_imshow(result2)                    
        #writer.write(result1)
        #writer.write(result2)

        currentFrame += 1
        bar.update(currentFrame)
      
      else:
          grabbed = False

    #cv2_imshow(result)
        
    #writer.release()
    vs.release()
    bar.finish()

    end = time.time()
    elap = (end - start)
    print("[INFO] process took {:.4f} seconds".format(elap))

   #print("Video created")
    response = requests.get('http://ai-analytics.asia/img/')

    return {"prediction": "response.ok"}


  except:
        my_logger.error("Something went wrong!")
        return {"prediction": "error"}


'''

@app.route('/analysis/team/{team_id}/video/{video_id}}', methods=['POST'])
def get_color2():
    try:
        data = dict(iris)['data']
        print(data)
        iris_types = {
            0: 'setosa',
            1: 'versicolor',
            2: 'virginica'
        }
        prediction = list(map(lambda x: iris_types[x], model.predict(data).tolist()))
        log_proba = model.predict_log_proba(data).tolist()
        return {"prediction": prediction, "log_proba": log_proba}
    except:
        my_logger.error("Something went wrong!")
        return {"prediction": "error"}


    #return render_template('result.html')'''
    
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://948221318f20.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [01/Jun/2021 07:16:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2021 07:16:16] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [01/Jun/2021 07:16:17] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Jun/2021 07:17:11] "POST /get_color HTTP/1.1" 404 -
127.0.0.1 - - [01/Jun/2021 07:17:13] "GET /get_color HTTP/1.1" 404 -
127.0.0.1 - - [01/Jun/2021 07:21:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2021 07:21:24] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [01/Jun/2021 07:22:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2021 07:22:42] "GET /static/style.css HTTP/1.1" 404 -


In [ ]:

import requests
#app = Flask(__name__)
#run_with_ngrok(app)  

app = Flask(__name__, template_folder='/content/drive/My Drive/basketball/Project 3/')
run_with_ngrok(app)  
@app.route('/')
def home():
    return render_template("index.html")

@app.route('/game_analysis', methods=['POST'])
def game_analysis():
  try:
    get_color = [x for x in request.form.values()]
    print (get_color)

    path = get_color[2]
    print (path)

    src_pts = np.array([
        [320,330],
            [1100,330],
            [1300,550],
            [-10,550],

            
        ])    
    src_pts2 = np.array([
        [320,280],
            [1100,330],
            [1340,560],
            [-40,520],
            
        ]) 
        
    video_id = 1 
    team_id = ""
    video_url = path 
    boundaries = src_pts


    
    '''src_pts = np.array([
        [320,330],
            [1100,330],
            [1300,550],
            [-10,550],

            
        ])    
    src_pts2 = np.array([
        [320,280],
            [1100,330],
            [1340,560],
            [-40,520],
            
        ]) 



        
    video_id = 1 
    team_id = ""
    video_url = path 
    boundaries = src_pts'''


    
    # Use the boxes info from the tensor prediction result
    #
    # x1,y1 ------
    # |          |
    # |          |
    # |          |
    # --------x2,y2
    #

   
    cfg = get_cfg()
    #cfg.merge_from_other_cfg
    cfg.merge_from_file("./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
    cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
    predictor = DefaultPredictor(cfg)

    def drawPlayers(im, pred_boxes, showResult=False):


      
      players_output = predictor(im)

      
      # look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
      instances = players_output["instances"]
      pred_boxes = instances.get("pred_boxes")
      pred_classes = instances.get("pred_classes")

      
      # We can use `Visualizer` to draw the predictions on the image.
      v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.0)
      v = v.draw_instance_predictions(players_output["instances"].to("cpu"))


      


      data = [[10,10],[12,12]]

      pos = pd.DataFrame(data,columns=['pos', 'time'])


      import time

      t = time.process_time()
      #elapsed_time = time.process_time() - t


      #rgb_color1 = input('Enter team colours ')
      rgb_color1 = name_to_rgb(get_color[0])
      #print(rgb_color1)
      c1 = np.uint8([[rgb_color1]]) #here insert the bgr values which you want to convert to hsv
      hsvcolour1 = cv2.cvtColor(c1, cv2.COLOR_BGR2HSV)
      lowerLimit1 = hsvcolour1[0][0][0] + 40, 80, 110
      upperLimit1 = hsvcolour1[0][0][0] + 70, 255, 255


      #get the second team colour
      rgb_color2 = name_to_rgb(get_color[1])
      #print(rgb_color2)
      c2 = np.uint8([[rgb_color2]]) #here insert the bgr values which you want to convert to hsv
      hsvcolour2 = cv2.cvtColor(c2, cv2.COLOR_BGR2HSV)
      lowerLimit2 = hsvcolour2[0][0][0] + 0, 0, 231
      upperLimit2 = hsvcolour2[0][0][0] + 180, 18, 255




      # Convert the imageFrame in
      # BGR(RGB color space) to
      # HSV(hue-saturation-value)
      # color space
      hsvFrame = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)

      #cv2_imshow(hsvFrame)

      # Set range for red color and
      # define mask
      #red_lower = np.array([136, 87, 111], np.uint8)
      #red_upper = np.array([180, 255, 255], np.uint8)

      lowerLimit1 = np.array(lowerLimit1, np.uint8)
      upperLimit1 = np.array(upperLimit1, np.uint8)
      print (lowerLimit1, upperLimit1 )
      team_mask1 = cv2.inRange(hsvFrame, lowerLimit1, upperLimit1)


      lowerLimit2 = np.array(lowerLimit2, np.uint8)
      upperLimit2 = np.array(upperLimit2, np.uint8)
      print (lowerLimit2, upperLimit2 )
      team_mask2 = cv2.inRange(hsvFrame, lowerLimit2, upperLimit2)



      # Morphological Transform, Dilation
      # for each color and bitwise_and operator
      # between imageFrame and mask determines
      # to detect only that particular color
      kernal = np.ones((5, 5), "uint8")

      # For red color
      team_mask1 = cv2.dilate(team_mask1, kernal)
      res_red = cv2.bitwise_and(im, im,
                  mask = team_mask1)

      # For black color
      team_mask2 = cv2.dilate(team_mask2, kernal)
      res_black = cv2.bitwise_and(im, im,
                  mask = team_mask2)




      f = open('ab.txt', 'a')
      color1 = [0, 0, 255] 
      color2 = [0, 0, 0] 
      thickness = 5
      radius = 5

      

      i  = 0
      for box in pred_boxes:
        
        # Include only class Person
        if pred_classes[i] == 0:  
            
          x1 = int(box[0])
          y1 = int(box[1])
          x2 = int(box[2])
          y2 = int(box[3])

          xc = x1 + int((x2 - x1)/2)
          player_pos1 = (xc - 1, y2)
          player_pos2 = (xc + 1, y2 + 1)

          court = Polygon(src_pts)
          court2 = Polygon(src_pts2)

          # Draw only players that are within the basketball court
          if Point(player_pos1).within(court):
            if showResult:
              print(".")
            
            # Creating contour to track red color
            contours, hierarchy = cv2.findContours(team_mask1,
                      cv2.RETR_TREE,
                      cv2.CHAIN_APPROX_SIMPLE)
            for pic, contour in enumerate(contours):
              area = cv2.contourArea(contour)
              if (area > 300):
                x, y, w, h = cv2.boundingRect(contour)

                if Point((x,y)).within(court2):
                  #cv2.rectangle(im, (x,y),(x + w, y + h),  (0, 0, 255), 2)

                  if (x== x1) : 
                    print(x,x1)
                    cv2.rectangle(im, (x,y),(x + w, y + h), (0, 0, 255), 2) 
                    team_id = "a" 
                    #cv2.rectangle(im, player_pos1,player_pos2, (0, 0, 255), thickness)
                    #cv2.putText(im, "T1", (x,y),cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 0, 255))
                    elapsed_time = time.process_time() - t
                    print("Pos1 " + str(player_pos1) + "Pos2"+ str(player_pos2)+ "time "+str(elapsed_time)+ "color "+str(rgb_color1), file = f)
          
                  
                    
            # Creating contour to track black color
            contours, hierarchy = cv2.findContours(team_mask2,
              cv2.RETR_TREE,
              cv2.CHAIN_APPROX_SIMPLE)
            for pic, contour in enumerate(contours):
              area = cv2.contourArea(contour)
              if (area > 300):
                p, q, r, s = cv2.boundingRect(contour)
                #print(p,q,r,s)
                if Point((p,q)).within(court2):
                  if (p == x1) :
                    print(p,x1) 
                    cv2.rectangle(im, (p,q),(p + r, q + s), (255, 255, 255), 2) 
                    team_id = "b" 
                    #cv2.rectangle(im, player_pos1, player_pos2, (255, 0, 0), thickness)
                    #cv2.putText(im, "T2", player_pos1,cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255, 255, 255))
                    elapsed_time = time.process_time() - t
                    print("Pos1 " + str(player_pos1) + "Pos2"+ str(player_pos2)+ "time "+str(elapsed_time)+ "color "+str(rgb_color2), file = f)
                    
              

                    
            #elapsed_time = time.process_time() - t
            #print("Pos1 " + str(player_pos1) + "Pos2"+ str(player_pos2)+ "time "+str(elapsed_time), file = f)
          
            pos.append({'pos': player_pos1[0],'time': player_pos1[1]},ignore_index=True)
            #print(player_pos2)

            i = i + 1    

          
      
      
    
      if showResult:
        print ("<")
        #cv2_imshow(im)



      pos.to_csv('pos1.csv', index=False)
      f.close()




    



    import time
    import progressbar
    from time import sleep
    from collections import deque
    from detectron2.utils.visualizer import ColorMode
    from detectron2.utils.visualizer import GenericMask
    import imutils

    
    !wget http://ai-analytics.asia/vid/abc.mp4 -O video.mp4
    vs = cv2.VideoCapture("./video.mp4")
    #vs = cv2.VideoCapture("")
    totalFrames = int(vs.get(cv2.CAP_PROP_FRAME_COUNT))

    grabbed = True
    currentFrame = 0
    start = time.time()
    writer = None

    bar = progressbar.ProgressBar(maxval=totalFrames, \
          widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

    bar.start()

    
    white_color = (255,255,255)
    red_color = (0,0,255)


    # loop over frames from the video file stream (207)
    while grabbed and currentFrame<=5 :     
      
      # read the next frame from the file
      (grabbed, frame) = vs.read()

      #if writer is None:
        #fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        #writer = cv2.VideoWriter("output.mp4", fourcc, 24, (court_img.shape[1], court_img.shape[0]), True)

      if grabbed:
        print(currentFrame)

       
        players_output = predictor(frame)

      
        # Get player positions
        outputs = predictor(frame)  
        instances = outputs["instances"].to("cpu")
        boxes = instances.get("pred_boxes")

        #court = court_img.copy()

        # Draw players on video frame
        drawPlayers(frame, boxes, False)
        
                    
        #writer.write(result1)
        #writer.write(result2)

        currentFrame += 1
        bar.update(currentFrame)
      
      else:
          grabbed = False

    #cv2_imshow(result)
        
    #writer.release()
    vs.release()
    bar.finish()

    end = time.time()
    elap = (end - start)
    print("[INFO] process took {:.4f} seconds".format(elap))


   #print("Video created")
    response = requests.get('http://ai-analytics.asia/img')

    
    return {"prediction": response.status_code}


  except:
        #my_logger.error("Something went wrong!")
        return {"prediction": "error"}



@app.route('/analysis/team/{team_id}/video/{video_id}}', methods=['POST'])
def analysis():
    try:
       with open('ab.txt','r') as fd:
         r = fd.read()
         print(r)
       return {"prediction": "response.ok"}

    except:
        my_logger.error("Something went wrong!")
        return {"prediction": "error"}


    #return render_template('result.html')'''
    
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://f054e08d6935.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [02/Jun/2021 07:16:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2021 07:16:21] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [02/Jun/2021 07:16:22] "GET /favicon.ico HTTP/1.1" 404 -


In [2]:
from flask import Flask, jsonify, request #import objects from the Flask model
app = Flask(__name__) #define app using Flask

languages = [{'name' : 'JavaScript'}, {'name' : 'Python'}, {'name' : 'Ruby'}]

@app.route('/', methods=['GET'])
def test():
	return jsonify({'message' : 'It works!'})

@app.route('/lang', methods=['GET'])
def returnAll():
	return jsonify({'languages' : languages})

@app.route('/lang/<string:name>', methods=['GET'])
def returnOne(name):
	langs = [language for language in languages if language['name'] == name]
	return jsonify({'language' : langs[0]})

@app.route('/lang', methods=['POST'])
def addOne():
	language = {'name' : request.json['name']}
	languages.append(language)
	return jsonify({'languages' : languages})

if __name__ == '__main__':
	app.run(debug=True) #run app on port 8080 in debug modee

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
